김유연 week 5 미션 풀이
=============

#### Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt

#### Q1.

#### Hyperparameters

In [2]:
# Hyperparameter
input_size = 28 * 28
num_classes = 10
learning_rate = 0.01
training_epochs = 15
batch_size = 100

root = './data'


#### Load Data

In [3]:
# load data
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, drop_last=True)

#### Q2. 

#### Check Hardware

In [4]:
# check cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Parameter containing:
tensor([[-0.3620, -0.0503,  0.6341,  ..., -0.9945,  0.5104, -0.5631],
        [-0.4225, -0.0379, -0.5495,  ...,  0.6429,  0.4456,  0.4760],
        [-1.7069,  0.7860,  0.7407,  ..., -0.5484, -0.1528, -0.5731],
        ...,
        [ 0.3420,  0.5363,  1.3591,  ...,  0.5344, -0.3243, -0.7127],
        [ 0.7674,  0.9376, -0.3123,  ...,  1.1611, -0.6034, -0.9007],
        [-0.3012,  0.2841,  0.1790,  ...,  1.0155,  0.2228,  0.2085]],
       device='cuda:0', requires_grad=True)

#### Build and Initialize Model

In [ ]:
# build linear model
linear = nn.Linear(in_features=input_size, out_features=num_classes, bias=True).to(device)

# weight init
torch.nn.init.normal_(linear.weight)

#### Q3. Loss and optimizer

In [5]:
# Loss fn - Cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss().to(device)

# optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.1)

#### Q4. Train

In [6]:
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad() # optimizer zero grad
        loss.backward() # loss backward
        optimizer.step() # optimizer step

        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item() * 100))

Epoch [1/15], Step [100/600], Loss: 3.9115, Accuracy: 40.00%
Epoch [1/15], Step [200/600], Loss: 1.8503, Accuracy: 59.00%
Epoch [1/15], Step [300/600], Loss: 1.7527, Accuracy: 67.00%
Epoch [1/15], Step [400/600], Loss: 1.7146, Accuracy: 69.00%
Epoch [1/15], Step [500/600], Loss: 1.1771, Accuracy: 76.00%
Epoch [1/15], Step [600/600], Loss: 1.2811, Accuracy: 77.00%
Epoch [2/15], Step [100/600], Loss: 0.6582, Accuracy: 86.00%
Epoch [2/15], Step [200/600], Loss: 1.3995, Accuracy: 78.00%
Epoch [2/15], Step [300/600], Loss: 0.8733, Accuracy: 81.00%
Epoch [2/15], Step [400/600], Loss: 0.9580, Accuracy: 79.00%
Epoch [2/15], Step [500/600], Loss: 1.4337, Accuracy: 73.00%
Epoch [2/15], Step [600/600], Loss: 1.0620, Accuracy: 79.00%
Epoch [3/15], Step [100/600], Loss: 1.0108, Accuracy: 74.00%
Epoch [3/15], Step [200/600], Loss: 0.2869, Accuracy: 90.00%
Epoch [3/15], Step [300/600], Loss: 0.7097, Accuracy: 87.00%
Epoch [3/15], Step [400/600], Loss: 0.6137, Accuracy: 85.00%
Epoch [3/15], Step [500/

#### Q5. Test

In [7]:
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs)
        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()

    print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images: 89.36%
